# This notebook must be run with the Vizgen_2 conda environment within the Vizgen.sif singularity container

In [ ]:
import scanpy as sc
import loompy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import anndata

In [ ]:
base_path = 'Shapes_Spatial/'
metas = [
    '24-mo-female-1/',
    '24-mo-female-3/',
    '24-mo-female-5/',
    '24-mo-male-1/',
    '24-mo-male-2/',
    '24-mo-male-4-rev2/',
    '3-mo-male-1/',
    '3-mo-male-2/',
    '3-mo-male-3-rev2/',
    '3-mo-female-1-rev2/',
    '3-mo-female-2/',
    '3-mo-female-3/'
]
adatas = []
"""
prior to this step we manually placed the .loom outputs from Baysor into a folder called 'baysor/' contained within the
individual meta data folders

"""
for meta in metas:
    file_path = os.path.join(base_path, meta, 'baysor')
    loom_files = [f for f in os.listdir(file_path) if f.endswith('.loom') and '6-5' in f]
    if loom_files:
        loom_file = loom_files[0]  # Assuming there is only one .loom file (If you only ran one parameter of baysor)
        adata = sc.read_loom(os.path.join(file_path, loom_file))
        adata.obs['Age'] = meta.split('-')[0]
        adata.obs['Sex'] = meta.split('-')[2]
        adata.obs['batchID'] = meta.split('/')[0]
        adata.var_names = adata.var.Name
        adata.obsm['spatial'] = adata.obs[["x", "y"]].values
        adatas.append(adata)

In [ ]:
combined_adata = anndata.concat(adatas, axis=0)
combined_adata = combined_adata[combined_adata.obs.n_transcripts > 20]

combined_adata.layers['counts'] = combined_adata.X

sc.pp.normalize_total(combined_adata)
sc.pp.log1p(combined_adata)
sc.pp.scale(combined_adata, max_value=10)
sc.tl.pca(combined_adata, svd_solver='arpack')
sc.external.pp.harmony_integrate(combined_adata, key='batchID')
print('Batch correction (Harmony) completed.')

sc.pp.neighbors(combined_adata)
sc.tl.umap(combined_adata)
sc.tl.leiden(combined_adata, resolution=1)
combined_adata.obsm['spatial'] = combined_adata.obs[["x", "y"]].values
combined_adata.write('baysor_segmented_6-5_micron.h5ad')